In [ ]:
from_lang = 'pt'
def sentiment_score(sentence):
    try:
        # api call
        to_lang = 'en'
        api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(sentence, from_lang,
                                                                                                  to_lang)
        hdrs = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
        response = requests.get(api_url, headers=hdrs)
        response_json = json.loads(response.text)

        # translation
        translation = response_json["responseData"]["translatedText"]

        # sentiment
        analyzer = SentimentIntensityAnalyzer()
        compound_score = analyzer.polarity_scores(translation)['compound']
        return [sentence, translation, compound_score]
    except Exception as e:
        print(str(e))
        return None
    
loop = asyncio.get_event_loop()

In [ ]:
async def get_sentiments():
    messages = list(table.message)
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        loop = asyncio.get_event_loop()
        futures = [loop.run_in_executor(
            executor,
            partial(
                sentiment_score,
                sentence=message
            ),
        ) for message in messages[0:100] ]
        
        for result in await asyncio.gather(*futures):
            print(result)

asyncio.run_coroutine_threadsafe(get_sentiments(), loop)